# Task 4.4: Normalization Comparison

**Module:** 4 - Neural Network Fundamentals  
**Time:** 2 hours  
**Difficulty:** ⭐⭐⭐ (Intermediate)

---

## 🎯 Learning Objectives

By the end of this notebook, you will:
- [ ] Implement BatchNorm from scratch (forward + backward)
- [ ] Implement LayerNorm from scratch
- [ ] Understand RMSNorm (used in modern LLMs)
- [ ] Compare training stability and convergence
- [ ] Know when to use each normalization technique

---

## 📚 Prerequisites

- Completed: Notebooks 01-03
- Knowledge of: Mean, variance, neural network training

---

## 🌍 Real-World Context

**Normalization is essential in modern deep learning:**

- **Batch Normalization (2015)**: Revolutionized CNN training, enabled 10x deeper networks
- **Layer Normalization (2016)**: Made transformers possible by removing batch dependency
- **RMSNorm (2019)**: Powers LLaMA, Mistral, and modern LLMs - faster than LayerNorm!

Without normalization, training deep networks would be nearly impossible.

---

## 🧒 ELI5: What is Normalization?

> **Imagine you're comparing test scores from different countries.**
>
> Country A scores: 80-100 (high average, narrow range)  
> Country B scores: 20-80 (lower average, wide range)  
>
> How do you compare fairly? **You normalize!**
> 1. Subtract the mean (center at zero)
> 2. Divide by standard deviation (make scales comparable)
>
> Now all scores are on the same scale, centered around 0 with similar spread.
>
> **In neural networks**, activations can have wildly different scales across layers. Normalization keeps everything "fair" so learning happens smoothly.
>
> The difference between BatchNorm and LayerNorm is just **which scores we compare**:
> - **BatchNorm**: Compare students across schools (normalize across batch)
> - **LayerNorm**: Compare each student across subjects (normalize across features)

---

## Setup

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from typing import Tuple, List, Dict, Optional
import time
import sys
import os

sys.path.insert(0, os.path.join(os.path.dirname(os.getcwd()), 'scripts'))

np.random.seed(42)
plt.style.use('default')
%matplotlib inline

print("✅ Setup complete!")

---

## Part 1: Implementing Batch Normalization

### The Algorithm

For a batch of activations, BatchNorm computes:

1. **Mean** across batch: $\mu = \frac{1}{m} \sum_i x_i$
2. **Variance** across batch: $\sigma^2 = \frac{1}{m} \sum_i (x_i - \mu)^2$
3. **Normalize**: $\hat{x}_i = \frac{x_i - \mu}{\sqrt{\sigma^2 + \epsilon}}$
4. **Scale and shift**: $y_i = \gamma \hat{x}_i + \beta$

Where $\gamma$ and $\beta$ are learnable parameters!

In [ ]:
class BatchNorm:
    """
    Batch Normalization layer.
    
    Normalizes across the batch dimension.
    
    Key insight: During training, use batch statistics.
                 During inference, use running (moving average) statistics.
    """
    
    def __init__(self, num_features: int, momentum: float = 0.1, epsilon: float = 1e-5):
        """
        Args:
            num_features: Number of features (neurons in previous layer)
            momentum: For running statistics update (0 = no update, 1 = replace)
            epsilon: For numerical stability
        """
        self.num_features = num_features
        self.momentum = momentum
        self.epsilon = epsilon
        
        # Learnable parameters
        self.gamma = np.ones(num_features)   # Scale
        self.beta = np.zeros(num_features)   # Shift
        
        # Running statistics for inference
        self.running_mean = np.zeros(num_features)
        self.running_var = np.ones(num_features)
        
        # Gradients
        self.dgamma = None
        self.dbeta = None
        
        # Cache
        self.cache = {}
        self.training = True
    
    def forward(self, x: np.ndarray) -> np.ndarray:
        """
        Forward pass.
        
        Args:
            x: Input of shape (batch_size, num_features)
        """
        if self.training:
            # Compute batch statistics
            batch_mean = np.mean(x, axis=0)
            batch_var = np.var(x, axis=0)
            
            # Normalize
            x_centered = x - batch_mean
            std = np.sqrt(batch_var + self.epsilon)
            x_norm = x_centered / std
            
            # Update running statistics
            self.running_mean = (1 - self.momentum) * self.running_mean + self.momentum * batch_mean
            self.running_var = (1 - self.momentum) * self.running_var + self.momentum * batch_var
            
            # Cache for backward
            self.cache['x'] = x
            self.cache['x_norm'] = x_norm
            self.cache['mean'] = batch_mean
            self.cache['var'] = batch_var
            self.cache['std'] = std
            self.cache['x_centered'] = x_centered
        else:
            # Use running statistics during inference
            x_norm = (x - self.running_mean) / np.sqrt(self.running_var + self.epsilon)
        
        # Scale and shift
        out = self.gamma * x_norm + self.beta
        return out
    
    def backward(self, dout: np.ndarray) -> np.ndarray:
        """
        Backward pass.
        
        This is the tricky part! The gradient flows through the normalization,
        which depends on batch statistics.
        """
        x = self.cache['x']
        x_norm = self.cache['x_norm']
        x_centered = self.cache['x_centered']
        std = self.cache['std']
        var = self.cache['var']
        m = x.shape[0]  # batch size
        
        # Gradients of learnable parameters
        self.dgamma = np.sum(dout * x_norm, axis=0)
        self.dbeta = np.sum(dout, axis=0)
        
        # Gradient through scale
        dx_norm = dout * self.gamma
        
        # Gradient through normalization (this is the complex part)
        # See: https://kevinzakka.github.io/2016/09/14/batch_normalization/
        dvar = np.sum(dx_norm * x_centered * -0.5 * (var + self.epsilon) ** (-1.5), axis=0)
        dmean = np.sum(dx_norm * -1 / std, axis=0) + dvar * np.mean(-2 * x_centered, axis=0)
        
        dx = dx_norm / std + dvar * 2 * x_centered / m + dmean / m
        
        return dx
    
    def __call__(self, x: np.ndarray) -> np.ndarray:
        return self.forward(x)

In [ ]:
# Test BatchNorm
print("🧪 Testing Batch Normalization")
print("=" * 50)

# Create input with non-zero mean and non-unit variance
x = np.random.randn(32, 256) * 5 + 3  # mean~3, std~5
print(f"Input stats:")
print(f"  Mean: {x.mean():.2f} (should be ~3)")
print(f"  Std:  {x.std():.2f} (should be ~5)")

# Apply BatchNorm
bn = BatchNorm(256)
out = bn(x)

print(f"\nOutput stats (per feature, averaged across batch):")
print(f"  Mean: {out.mean(axis=0).mean():.6f} (should be ~0)")
print(f"  Std:  {out.std(axis=0).mean():.2f} (should be ~1)")

# Test backward
dout = np.random.randn(32, 256)
dx = bn.backward(dout)
print(f"\nBackward pass: dx shape = {dx.shape}")
print(f"Gamma gradient shape: {bn.dgamma.shape}")

print("\n✅ BatchNorm test passed!")

---

## Part 2: Implementing Layer Normalization

### Key Difference from BatchNorm

- **BatchNorm**: Normalize across batch dimension (features are normalized together across samples)
- **LayerNorm**: Normalize across feature dimension (each sample is normalized independently)

In [ ]:
class LayerNorm:
    """
    Layer Normalization.
    
    Normalizes across the feature dimension (not batch).
    
    Key advantages over BatchNorm:
    - Works with batch_size=1
    - Same behavior in training and inference
    - Perfect for transformers and RNNs
    """
    
    def __init__(self, normalized_shape: int, epsilon: float = 1e-5):
        self.normalized_shape = normalized_shape
        self.epsilon = epsilon
        
        # Learnable parameters
        self.gamma = np.ones(normalized_shape)
        self.beta = np.zeros(normalized_shape)
        
        # Gradients
        self.dgamma = None
        self.dbeta = None
        
        # Cache
        self.cache = {}
    
    def forward(self, x: np.ndarray) -> np.ndarray:
        """
        Forward pass.
        
        Normalize each sample independently across features.
        """
        # Compute mean and variance across features (axis=-1)
        mean = np.mean(x, axis=-1, keepdims=True)
        var = np.var(x, axis=-1, keepdims=True)
        
        # Normalize
        std = np.sqrt(var + self.epsilon)
        x_norm = (x - mean) / std
        
        # Cache for backward
        self.cache['x'] = x
        self.cache['x_norm'] = x_norm
        self.cache['mean'] = mean
        self.cache['var'] = var
        self.cache['std'] = std
        
        # Scale and shift
        out = self.gamma * x_norm + self.beta
        return out
    
    def backward(self, dout: np.ndarray) -> np.ndarray:
        """
        Backward pass.
        """
        x = self.cache['x']
        x_norm = self.cache['x_norm']
        mean = self.cache['mean']
        var = self.cache['var']
        std = self.cache['std']
        n = x.shape[-1]  # Number of features
        
        # Gradients of learnable parameters
        self.dgamma = np.sum(dout * x_norm, axis=0)
        self.dbeta = np.sum(dout, axis=0)
        
        # Gradient through scale
        dx_norm = dout * self.gamma
        
        # Gradient through normalization
        x_centered = x - mean
        dvar = np.sum(dx_norm * x_centered * -0.5 * (var + self.epsilon) ** (-1.5), axis=-1, keepdims=True)
        dmean = np.sum(dx_norm * -1 / std, axis=-1, keepdims=True) + dvar * np.mean(-2 * x_centered, axis=-1, keepdims=True)
        
        dx = dx_norm / std + dvar * 2 * x_centered / n + dmean / n
        
        return dx
    
    def __call__(self, x: np.ndarray) -> np.ndarray:
        return self.forward(x)

In [ ]:
# Test LayerNorm
print("🧪 Testing Layer Normalization")
print("=" * 50)

x = np.random.randn(32, 256) * 5 + 3

ln = LayerNorm(256)
out = ln(x)

print(f"Output stats (per sample, should be normalized):")
print(f"  Mean per sample (first 5): {out.mean(axis=-1)[:5]}")
print(f"  Std per sample (first 5):  {out.std(axis=-1)[:5]}")

# Test backward
dout = np.random.randn(32, 256)
dx = ln.backward(dout)
print(f"\nBackward pass: dx shape = {dx.shape}")

print("\n✅ LayerNorm test passed!")

---

## Part 3: Understanding RMSNorm

### The Simplification

RMSNorm removes the mean-centering step:

$$\text{RMSNorm}(x) = \frac{x}{\text{RMS}(x)} \cdot \gamma$$

Where $\text{RMS}(x) = \sqrt{\frac{1}{n}\sum_i x_i^2}$

**Why it works:** For transformers, the mean-centering provides marginal benefit but adds computation. LLaMA showed RMSNorm works just as well and is faster!

In [ ]:
class RMSNorm:
    """
    Root Mean Square Layer Normalization.
    
    Used in LLaMA, Mistral, and other modern LLMs.
    Faster than LayerNorm (no mean computation), similar performance.
    """
    
    def __init__(self, normalized_shape: int, epsilon: float = 1e-5):
        self.normalized_shape = normalized_shape
        self.epsilon = epsilon
        
        # Only gamma (no beta in RMSNorm)
        self.gamma = np.ones(normalized_shape)
        
        # Gradients
        self.dgamma = None
        
        # Cache
        self.cache = {}
    
    def forward(self, x: np.ndarray) -> np.ndarray:
        """
        Forward pass.
        """
        # Compute RMS
        rms = np.sqrt(np.mean(x ** 2, axis=-1, keepdims=True) + self.epsilon)
        
        # Normalize
        x_norm = x / rms
        
        # Cache
        self.cache['x'] = x
        self.cache['rms'] = rms
        self.cache['x_norm'] = x_norm
        
        # Scale (no shift in RMSNorm)
        return self.gamma * x_norm
    
    def backward(self, dout: np.ndarray) -> np.ndarray:
        """
        Backward pass.
        """
        x = self.cache['x']
        rms = self.cache['rms']
        x_norm = self.cache['x_norm']
        n = x.shape[-1]
        
        # Gradient of gamma
        self.dgamma = np.sum(dout * x_norm, axis=0)
        
        # Gradient through scale
        dx_norm = dout * self.gamma
        
        # Gradient through RMS normalization
        drms = np.sum(dx_norm * x * -1 / (rms ** 2), axis=-1, keepdims=True)
        dx_sq = drms * 0.5 / rms / n
        
        dx = dx_norm / rms + 2 * dx_sq * x
        
        return dx
    
    def __call__(self, x: np.ndarray) -> np.ndarray:
        return self.forward(x)

In [ ]:
# Test RMSNorm
print("🧪 Testing RMSNorm")
print("=" * 50)

x = np.random.randn(32, 256) * 5 + 3

rms = RMSNorm(256)
out = rms(x)

# Check RMS of output
rms_values = np.sqrt(np.mean(out ** 2, axis=-1))
print(f"RMS per sample (should be ~1): {rms_values[:5]}")
print(f"Note: Mean is NOT centered (that's the point of RMSNorm)")
print(f"Mean per sample: {out.mean(axis=-1)[:5]}")

print("\n✅ RMSNorm test passed!")

---

## Part 4: Visual Comparison

Let's visualize what each normalization does to the data.

In [ ]:
# Create input with interesting statistics
np.random.seed(42)
x = np.random.randn(1000, 64) * 3 + 2  # Shifted and scaled

# Apply each normalization
bn = BatchNorm(64)
ln = LayerNorm(64)
rmsn = RMSNorm(64)

out_bn = bn(x)
out_ln = ln(x)
out_rms = rmsn(x)

# Visualize
fig, axes = plt.subplots(2, 4, figsize=(16, 8))

# Row 1: Distribution of a single feature across batch
feature_idx = 0

axes[0, 0].hist(x[:, feature_idx], bins=30, alpha=0.7, color='gray')
axes[0, 0].set_title(f'Original (Feature {feature_idx})')
axes[0, 0].set_xlabel('Value')

axes[0, 1].hist(out_bn[:, feature_idx], bins=30, alpha=0.7, color='blue')
axes[0, 1].set_title(f'After BatchNorm (Feature {feature_idx})')
axes[0, 1].axvline(0, color='red', linestyle='--')

axes[0, 2].hist(out_ln[:, feature_idx], bins=30, alpha=0.7, color='green')
axes[0, 2].set_title(f'After LayerNorm (Feature {feature_idx})')

axes[0, 3].hist(out_rms[:, feature_idx], bins=30, alpha=0.7, color='purple')
axes[0, 3].set_title(f'After RMSNorm (Feature {feature_idx})')

# Row 2: Distribution of a single sample across features
sample_idx = 0

axes[1, 0].hist(x[sample_idx], bins=30, alpha=0.7, color='gray')
axes[1, 0].set_title(f'Original (Sample {sample_idx})')
axes[1, 0].set_xlabel('Value')

axes[1, 1].hist(out_bn[sample_idx], bins=30, alpha=0.7, color='blue')
axes[1, 1].set_title(f'After BatchNorm (Sample {sample_idx})')

axes[1, 2].hist(out_ln[sample_idx], bins=30, alpha=0.7, color='green')
axes[1, 2].set_title(f'After LayerNorm (Sample {sample_idx})')
axes[1, 2].axvline(0, color='red', linestyle='--')

axes[1, 3].hist(out_rms[sample_idx], bins=30, alpha=0.7, color='purple')
axes[1, 3].set_title(f'After RMSNorm (Sample {sample_idx})')

plt.suptitle('Normalization Effects', fontsize=14)
plt.tight_layout()
plt.show()

print("\n💡 Key observations:")
print("   Row 1: BatchNorm normalizes across batch (each FEATURE is centered)")
print("   Row 2: LayerNorm normalizes across features (each SAMPLE is centered)")
print("   RMSNorm: Similar to LayerNorm but without centering")

---

## Part 5: Training Comparison

Let's compare training with different normalization methods.

In [ ]:
# Load MNIST
import gzip
import urllib.request

def load_mnist(path='../data'):
    os.makedirs(path, exist_ok=True)
    base_url = 'http://yann.lecun.com/exdb/mnist/'
    files = {
        'train_images': 'train-images-idx3-ubyte.gz',
        'train_labels': 'train-labels-idx1-ubyte.gz',
        'test_images': 't10k-images-idx3-ubyte.gz',
        'test_labels': 't10k-labels-idx1-ubyte.gz'
    }
    
    def download(filename):
        filepath = os.path.join(path, filename)
        if not os.path.exists(filepath):
            print(f"Downloading {filename}...")
            urllib.request.urlretrieve(base_url + filename, filepath)
        return filepath
    
    def load_images(fp):
        with gzip.open(fp, 'rb') as f:
            f.read(16)
            return np.frombuffer(f.read(), dtype=np.uint8).reshape(-1, 784).astype(np.float32) / 255.0
    
    def load_labels(fp):
        with gzip.open(fp, 'rb') as f:
            f.read(8)
            return np.frombuffer(f.read(), dtype=np.uint8)
    
    return (load_images(download(files['train_images'])),
            load_labels(download(files['train_labels'])),
            load_images(download(files['test_images'])),
            load_labels(download(files['test_labels'])))

X_train, y_train, X_test, y_test = load_mnist()
X_train, y_train = X_train[:5000], y_train[:5000]  # Subset for speed
print(f"Training samples: {len(X_train)}")

In [ ]:
class NormalizedMLP:
    """
    MLP with configurable normalization.
    """
    
    def __init__(self, layer_sizes: List[int], norm_type: Optional[str] = None):
        """
        Args:
            layer_sizes: e.g., [784, 256, 128, 10]
            norm_type: 'batch', 'layer', 'rms', or None
        """
        self.norm_type = norm_type
        self.training = True
        
        self.layers = []
        self.norms = []
        
        for i in range(len(layer_sizes) - 1):
            W = np.random.randn(layer_sizes[i], layer_sizes[i + 1]) * np.sqrt(2.0 / layer_sizes[i])
            b = np.zeros(layer_sizes[i + 1])
            self.layers.append({'W': W, 'b': b, 'cache': {}, 'dW': None, 'db': None})
            
            # Add normalization for hidden layers
            if i < len(layer_sizes) - 2 and norm_type:
                if norm_type == 'batch':
                    self.norms.append(BatchNorm(layer_sizes[i + 1]))
                elif norm_type == 'layer':
                    self.norms.append(LayerNorm(layer_sizes[i + 1]))
                elif norm_type == 'rms':
                    self.norms.append(RMSNorm(layer_sizes[i + 1]))
            else:
                self.norms.append(None)
    
    def forward(self, X: np.ndarray) -> np.ndarray:
        out = X
        
        for i, layer in enumerate(self.layers[:-1]):
            layer['cache']['X'] = out
            out = out @ layer['W'] + layer['b']
            layer['cache']['Z'] = out
            
            # Apply normalization if present
            if self.norms[i] is not None:
                if hasattr(self.norms[i], 'training'):
                    self.norms[i].training = self.training
                out = self.norms[i](out)
            
            # ReLU
            out = np.maximum(0, out)
        
        # Output layer
        self.layers[-1]['cache']['X'] = out
        out = out @ self.layers[-1]['W'] + self.layers[-1]['b']
        
        # Softmax
        out_shifted = out - np.max(out, axis=1, keepdims=True)
        exp_out = np.exp(out_shifted)
        self.probs = exp_out / np.sum(exp_out, axis=1, keepdims=True)
        
        return self.probs
    
    def backward(self, targets: np.ndarray, lr: float):
        batch_size = len(targets)
        grad = self.probs.copy()
        grad[np.arange(batch_size), targets] -= 1
        
        for i in range(len(self.layers) - 1, -1, -1):
            layer = self.layers[i]
            X = layer['cache']['X']
            
            layer['dW'] = X.T @ grad / batch_size
            layer['db'] = np.mean(grad, axis=0)
            grad = grad @ layer['W'].T
            
            if i > 0:
                Z = self.layers[i - 1]['cache']['Z']
                
                # ReLU backward
                grad = grad * (Z > 0).astype(float)
                
                # Normalization backward
                if self.norms[i - 1] is not None:
                    grad = self.norms[i - 1].backward(grad)
            
            layer['W'] -= lr * layer['dW']
            layer['b'] -= lr * layer['db']
            
            # Update norm parameters
            if i > 0 and self.norms[i - 1] is not None:
                norm = self.norms[i - 1]
                norm.gamma -= lr * norm.dgamma
                if hasattr(norm, 'beta') and norm.dbeta is not None:
                    norm.beta -= lr * norm.dbeta
    
    def predict(self, X: np.ndarray) -> np.ndarray:
        self.training = False
        probs = self.forward(X)
        self.training = True
        return np.argmax(probs, axis=1)

In [ ]:
def train_and_evaluate(norm_type, epochs=20, lr=0.1):
    np.random.seed(42)
    model = NormalizedMLP([784, 256, 128, 10], norm_type=norm_type)
    
    history = {'loss': [], 'acc': []}
    batch_size = 64
    
    for epoch in range(epochs):
        indices = np.random.permutation(len(X_train))
        epoch_loss = 0
        n_batches = 0
        
        for start in range(0, len(X_train), batch_size):
            batch_idx = indices[start:start + batch_size]
            X_batch = X_train[batch_idx]
            y_batch = y_train[batch_idx]
            
            probs = model.forward(X_batch)
            loss = -np.mean(np.log(probs[np.arange(len(y_batch)), y_batch] + 1e-10))
            model.backward(y_batch, lr)
            
            epoch_loss += loss
            n_batches += 1
        
        acc = np.mean(model.predict(X_test[:1000]) == y_test[:1000])
        history['loss'].append(epoch_loss / n_batches)
        history['acc'].append(acc)
    
    return history

In [ ]:
# Train with each normalization type
print("🏋️ Training with different normalization methods...")
print("=" * 60)

norm_types = [None, 'batch', 'layer', 'rms']
results = {}

for norm_type in norm_types:
    name = norm_type if norm_type else 'None'
    start = time.time()
    history = train_and_evaluate(norm_type, epochs=20)
    elapsed = time.time() - start
    results[name] = history
    print(f"{name:10s} | Final Acc: {history['acc'][-1]:.2%} | Time: {elapsed:.2f}s")

In [ ]:
# Plot comparison
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

colors = {'None': 'gray', 'batch': 'blue', 'layer': 'green', 'rms': 'purple'}

for name, history in results.items():
    axes[0].plot(history['loss'], color=colors[name], linewidth=2, label=name)
    axes[1].plot(history['acc'], color=colors[name], linewidth=2, label=name)

axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Loss')
axes[0].set_title('Training Loss')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Test Accuracy')
axes[1].set_title('Test Accuracy')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

---

## Part 6: Summary and Recommendations

In [ ]:
print("\n" + "=" * 80)
print("                    NORMALIZATION COMPARISON SUMMARY")
print("=" * 80)

print("""
┌──────────────┬─────────────────────┬─────────────────────────────────────────┐
│ Method       │ Normalizes Over     │ Best For                                │
├──────────────┼─────────────────────┼─────────────────────────────────────────┤
│ BatchNorm    │ Batch dimension     │ CNNs, large batches, computer vision    │
│              │ (across samples)    │ Different behavior train vs inference   │
├──────────────┼─────────────────────┼─────────────────────────────────────────┤
│ LayerNorm    │ Feature dimension   │ Transformers, RNNs, NLP                 │
│              │ (within each sample)│ Same behavior train and inference       │
├──────────────┼─────────────────────┼─────────────────────────────────────────┤
│ RMSNorm      │ Feature dimension   │ Modern LLMs (LLaMA, Mistral)            │
│              │ (no mean centering) │ Faster than LayerNorm, similar quality  │
└──────────────┴─────────────────────┴─────────────────────────────────────────┘

Key Decision Factors:
1. Batch Size: Small/variable → LayerNorm or RMSNorm
2. Architecture: CNN → BatchNorm, Transformer → LayerNorm/RMSNorm
3. Speed Priority: RMSNorm (no mean computation)
4. Train/Inference Consistency: LayerNorm or RMSNorm
""")

---

## ⚠️ Common Mistakes

### Mistake 1: Using BatchNorm with batch_size=1

```python
# ❌ Wrong - BatchNorm needs multiple samples for statistics
bn = BatchNorm(256)
out = bn(single_sample)  # Only one sample!

# ✅ Right - Use LayerNorm for single samples
ln = LayerNorm(256)
out = ln(single_sample)
```

### Mistake 2: Forgetting to switch training mode

```python
# ❌ Wrong - Using batch statistics during inference
model.eval()  # But BatchNorm inside still uses batch stats!

# ✅ Right - Set all BatchNorm layers to eval mode
for layer in model.layers:
    if isinstance(layer, BatchNorm):
        layer.training = False
```

---

## 🎉 Checkpoint

You've learned:

- ✅ Implemented BatchNorm with forward and backward passes
- ✅ Implemented LayerNorm from scratch
- ✅ Understood RMSNorm and why LLMs use it
- ✅ Compared training stability with different normalizations
- ✅ Know when to use each technique

---

## 📖 Further Reading

- [Batch Normalization Paper](https://arxiv.org/abs/1502.03167)
- [Layer Normalization Paper](https://arxiv.org/abs/1607.06450)
- [RMSNorm Paper](https://arxiv.org/abs/1910.07467)

---

## 🧹 Cleanup

In [ ]:
import gc
gc.collect()

print("✅ Cleanup complete!")
print("\n🎯 Next: Proceed to notebook 05-training-diagnostics-lab.ipynb")